# Sampler Metrics

Here we will get the metrics of our successfull samplers that sampled from the posterior of MHNs trained on the LUAD and COAD datasets with the symsparse penalty.
The results produced Table 1.

In [2]:
from mhn.mcmc.mcmc import MCMC
import mhn
import numpy as np
import pandas as pd

burn_in = 0.2

In [6]:
info = dict()

for data_name in ["G13_LUAD_12", "G13_COAD_12"]:
    model = mhn.model.oMHN.load(f"../results/mhns/{data_name}_symsparse_mle.csv")
    data = np.loadtxt(f"../data/{data_name}.csv", delimiter=",",
                      skiprows=1,
                      usecols=np.arange(1, model.log_theta.shape[1] + 1))
    mcmc = MCMC(mhn_model=model, data=data,
                penalty=mhn.optimizers.Penalty.SYM_SPARSE)
    mcmc.log_thetas = np.load(
        "../results/mcmc/G13_LUAD_12_symsparse_MALA_0_001421875.npy"
        if data_name == "G13_LUAD_12"
        else "../results/mcmc/G13_COAD_12_symsparse_MALA_0_001.npy"
        if data_name == "G13_COAD_12"
        else None
    )

    rhat = mcmc.rhat(burn_in=burn_in)
    ess = mcmc.ess(burn_in=burn_in)

    info[data_name] = {
        "dataset": data_name.split("_")[1],
        "essmin": np.min(ess),
        "essmedian": np.median(ess),
        "essmax": np.max(ess),
        "rhatmin": np.min(rhat),
        "rhatmedian": np.median(rhat),
        "rhatmax": np.max(rhat),
        "nsteps": int(mcmc.log_thetas.shape[1] *100 * burn_in),
    }

/home/lhu/learnmhn/mhn/optimizers.py:289: UserWarning: The dtype of the given data matrix is changed to np.int32
  warnings.warn(
/home/lhu/learnmhn/mhn/optimizers.py:289: UserWarning: The dtype of the given data matrix is changed to np.int32
  warnings.warn(


In [15]:
info_df = pd.DataFrame.from_dict(info, orient="index")
for col in info_df.columns:
    if col.startswith("ess"):
        info_df[col] = info_df[col].apply('{:.2f}'.format)
    if col.startswith("rhat"):
        info_df[col] = info_df[col].apply('{:.4f}'.format)
info_df
info_df.to_csv("../results/mcmc/symsparse_metrics.csv", index=False)